# 10 Minutes to cuDF and CuPy

This notebook provides introductory examples of how you can use cuDF and CuPy together to take advantage of CuPy array functionality (such as advanced linear algebra operations).

In [1]:
import timeit
from packaging import version

import cupy as cp
import cudf

if version.parse(cp.__version__) >= version.parse("10.0.0"):
    cupy_from_dlpack = cp.from_dlpack
else:
    cupy_from_dlpack = cp.fromDlpack

### Converting a cuDF DataFrame to a CuPy Array

If we want to convert a cuDF DataFrame to a CuPy ndarray, There are multiple ways to do it:

1. We can use the [dlpack](https://github.com/dmlc/dlpack) interface.

2. We can also use `DataFrame.values`.

3. We can also convert via the [CUDA array interface](https://numba.pydata.org/numba-doc/dev/cuda/cuda_array_interface.html) by using cuDF's `as_gpu_matrix` and CuPy's `asarray` functionality.

In [2]:
nelem = 10000
df = cudf.DataFrame({'a':range(nelem),
                     'b':range(500, nelem + 500),
                     'c':range(1000, nelem + 1000)}
                   )

%timeit arr_cupy = cupy_from_dlpack(df.to_dlpack())
%timeit arr_cupy = df.values
%timeit arr_cupy = df.to_cupy()

183 µs ± 1.15 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
553 µs ± 6.25 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
546 µs ± 2.25 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [3]:
arr_cupy = cupy_from_dlpack(df.to_dlpack())
arr_cupy

array([[    0,   500,  1000],
       [    1,   501,  1001],
       [    2,   502,  1002],
       ...,
       [ 9997, 10497, 10997],
       [ 9998, 10498, 10998],
       [ 9999, 10499, 10999]])

### Converting a cuDF Series to a CuPy Array

There are also multiple ways to convert a cuDF Series to a CuPy array:

1. We can pass the Series to `cupy.asarray` as cuDF Series exposes [`__cuda_array_interface__`](https://docs-cupy.chainer.org/en/stable/reference/interoperability.html).
2. We can leverage the dlpack interface `to_dlpack()`. 
3. We can also use `Series.values` 


In [4]:
col = 'a'

%timeit cola_cupy = cp.asarray(df[col])
%timeit cola_cupy = cupy_from_dlpack(df[col].to_dlpack())
%timeit cola_cupy = df[col].values

76.8 µs ± 636 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
198 µs ± 2.72 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
181 µs ± 1.1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [5]:
cola_cupy = cp.asarray(df[col])
cola_cupy

array([   0,    1,    2, ..., 9997, 9998, 9999])

From here, we can proceed with normal CuPy workflows, such as reshaping the array, getting the diagonal, or calculating the norm.

In [6]:
reshaped_arr = cola_cupy.reshape(50, 200)
reshaped_arr

array([[   0,    1,    2, ...,  197,  198,  199],
       [ 200,  201,  202, ...,  397,  398,  399],
       [ 400,  401,  402, ...,  597,  598,  599],
       ...,
       [9400, 9401, 9402, ..., 9597, 9598, 9599],
       [9600, 9601, 9602, ..., 9797, 9798, 9799],
       [9800, 9801, 9802, ..., 9997, 9998, 9999]])

In [7]:
reshaped_arr.diagonal()

array([   0,  201,  402,  603,  804, 1005, 1206, 1407, 1608, 1809, 2010,
       2211, 2412, 2613, 2814, 3015, 3216, 3417, 3618, 3819, 4020, 4221,
       4422, 4623, 4824, 5025, 5226, 5427, 5628, 5829, 6030, 6231, 6432,
       6633, 6834, 7035, 7236, 7437, 7638, 7839, 8040, 8241, 8442, 8643,
       8844, 9045, 9246, 9447, 9648, 9849])

In [8]:
cp.linalg.norm(reshaped_arr)

array(577306.967739)

### Converting a CuPy Array to a cuDF DataFrame

We can also convert a CuPy ndarray to a cuDF DataFrame. Like before, there are multiple ways to do it:

1. **Easiest;** We can directly use the `DataFrame` constructor.

2. We can use CUDA array interface with the `DataFrame` constructor.

3. We can also use the [dlpack](https://github.com/dmlc/dlpack) interface.

For the latter two cases, we'll need to make sure that our CuPy array is Fortran contiguous in memory (if it's not already). We can either transpose the array or simply coerce it to be Fortran contiguous beforehand.

In [9]:
%timeit reshaped_df = cudf.DataFrame(reshaped_arr)

23.9 ms ± 119 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
reshaped_df = cudf.DataFrame(reshaped_arr)
reshaped_df.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
1,200,201,202,203,204,205,206,207,208,209,...,390,391,392,393,394,395,396,397,398,399
2,400,401,402,403,404,405,406,407,408,409,...,590,591,592,593,594,595,596,597,598,599
3,600,601,602,603,604,605,606,607,608,609,...,790,791,792,793,794,795,796,797,798,799
4,800,801,802,803,804,805,806,807,808,809,...,990,991,992,993,994,995,996,997,998,999


We can check whether our array is Fortran contiguous by using cupy.isfortran or looking at the [flags](https://docs-cupy.chainer.org/en/stable/reference/generated/cupy.ndarray.html#cupy.ndarray.flags) of the array.

In [11]:
cp.isfortran(reshaped_arr)

False

In this case, we'll need to convert it before going to a cuDF DataFrame. In the next two cells, we create the DataFrame by leveraging dlpack and the CUDA array interface, respectively.

In [12]:
%%timeit

fortran_arr = cp.asfortranarray(reshaped_arr)
reshaped_df = cudf.DataFrame(fortran_arr)

9.15 ms ± 131 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%%timeit

fortran_arr = cp.asfortranarray(reshaped_arr)
reshaped_df = cudf.from_dlpack(fortran_arr.toDlpack())

5.74 ms ± 29.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
fortran_arr = cp.asfortranarray(reshaped_arr)
reshaped_df = cudf.DataFrame(fortran_arr)
reshaped_df.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
1,200,201,202,203,204,205,206,207,208,209,...,390,391,392,393,394,395,396,397,398,399
2,400,401,402,403,404,405,406,407,408,409,...,590,591,592,593,594,595,596,597,598,599
3,600,601,602,603,604,605,606,607,608,609,...,790,791,792,793,794,795,796,797,798,799
4,800,801,802,803,804,805,806,807,808,809,...,990,991,992,993,994,995,996,997,998,999


### Converting a CuPy Array to a cuDF Series

To convert an array to a Series, we can directly pass the array to the `Series` constructor.

In [15]:
cudf.Series(reshaped_arr.diagonal()).head()

0      0
1    201
2    402
3    603
4    804
dtype: int64

### Interweaving CuDF and CuPy for Smooth PyData Workflows

RAPIDS libraries and the entire GPU PyData ecosystem are developing quickly, but sometimes a one library may not have the functionality you need. One example of this might be taking the row-wise sum (or mean) of a Pandas DataFrame. cuDF's support for row-wise operations isn't mature, so you'd need to either transpose the DataFrame or write a UDF and explicitly calculate the sum across each row. Transposing could lead to hundreds of thousands of columns (which cuDF wouldn't perform well with) depending on your data's shape, and writing a UDF can be time intensive.

By leveraging the interoperability of the GPU PyData ecosystem, this operation becomes very easy. Let's take the row-wise sum of our previously reshaped cuDF DataFrame.

In [16]:
reshaped_df.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
1,200,201,202,203,204,205,206,207,208,209,...,390,391,392,393,394,395,396,397,398,399
2,400,401,402,403,404,405,406,407,408,409,...,590,591,592,593,594,595,596,597,598,599
3,600,601,602,603,604,605,606,607,608,609,...,790,791,792,793,794,795,796,797,798,799
4,800,801,802,803,804,805,806,807,808,809,...,990,991,992,993,994,995,996,997,998,999


We can just transform it into a CuPy array and use the `axis` argument of `sum`.

In [17]:
new_arr = cupy_from_dlpack(reshaped_df.to_dlpack())
new_arr.sum(axis=1)

array([  19900,   59900,   99900,  139900,  179900,  219900,  259900,
        299900,  339900,  379900,  419900,  459900,  499900,  539900,
        579900,  619900,  659900,  699900,  739900,  779900,  819900,
        859900,  899900,  939900,  979900, 1019900, 1059900, 1099900,
       1139900, 1179900, 1219900, 1259900, 1299900, 1339900, 1379900,
       1419900, 1459900, 1499900, 1539900, 1579900, 1619900, 1659900,
       1699900, 1739900, 1779900, 1819900, 1859900, 1899900, 1939900,
       1979900])

With just that single line, we're able to seamlessly move between data structures in this ecosystem, giving us enormous flexibility without sacrificing speed.

### Converting a cuDF DataFrame to a CuPy Sparse Matrix

We can also convert a DataFrame or Series to a CuPy sparse matrix. We might want to do this if downstream processes expect CuPy sparse matrices as an input.

The sparse matrix data structure is defined by three dense arrays. We'll define a small helper function for cleanliness.

In [18]:
def cudf_to_cupy_sparse_matrix(data, sparseformat='column'):
    """Converts a cuDF object to a CuPy Sparse Column matrix.
    """
    if sparseformat not in ('row', 'column',):
        raise ValueError("Let's focus on column and row formats for now.")
    
    _sparse_constructor = cp.sparse.csc_matrix
    if sparseformat == 'row':
        _sparse_constructor = cp.sparse.csr_matrix

    return _sparse_constructor(cupy_from_dlpack(data.to_dlpack()))

We can define a sparsely populated DataFrame to illustrate this conversion to either sparse matrix format.

In [19]:
df = cudf.DataFrame()
nelem = 10000
nonzero = 1000
for i in range(20):
    arr = cp.random.normal(5, 5, nelem)
    arr[cp.random.choice(arr.shape[0], nelem-nonzero, replace=False)] = 0
    df['a' + str(i)] = arr

In [20]:
df.head()

,a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16,a17,a18,a19
0,0.000000,0.0,0.0,0.000000,0.0,9.37476,0.000000,0.0,0.0,0.000000,6.237859,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000000
1,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.065878,0.0,0.0,12.35705,0.0,0.0,0.000000
2,3.232751,0.0,0.0,0.000000,0.0,0.00000,8.341915,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,3.110362
3,0.000000,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000000
4,0.000000,0.0,0.0,7.743024,0.0,0.00000,0.000000,0.0,0.0,5.987098,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000000


In [21]:
sparse_data = cudf_to_cupy_sparse_matrix(df)
print(sparse_data)

  (2, 0)	3.2327506467190874
  (259, 0)	10.723428115951062
  (643, 0)	0.47763624588488707
  (899, 0)	8.857065309921685
  (516, 0)	8.792407143276648
  (262, 0)	2.1900894573805396
  (390, 0)	5.007630701229646
  (646, 0)	6.630703075588639
  (392, 0)	5.573713453854357
  (776, 0)	10.501281989515688
  (904, 0)	8.261890175181366
  (1033, 0)	-0.41106824704220446
  (522, 0)	12.619952511457068
  (139, 0)	12.753348070606792
  (141, 0)	4.936902335394504
  (270, 0)	-1.7695949916946174
  (782, 0)	4.378746787324408
  (15, 0)	8.554141682891935
  (527, 0)	5.1994882136423
  (912, 0)	2.6101212854793125
  (401, 0)	5.614628764689268
  (403, 0)	9.999468341523317
  (787, 0)	7.6170790481600985
  (404, 0)	5.105328903336744
  (916, 0)	1.395526391114967
  :	:
  (9328, 19)	5.938629381103238
  (9457, 19)	4.463547879031807
  (9458, 19)	-0.8034946631917106
  (8051, 19)	-1.904327616912268
  (8819, 19)	8.314944347687199
  (7543, 19)	1.4303204025224376
  (8824, 19)	5.1559713157589
  (7673, 19)	7.478681299798863
  (7802,

From here, we could continue our workflow with a CuPy sparse matrix.

For a full list of the functionality built into these libraries, we encourage you to check out the API docs for [cuDF](https://docs.rapids.ai/api/cudf/nightly/) and [CuPy](https://docs-cupy.chainer.org/en/stable/index.html).